In [52]:
#!pip install pyreadstat
import pyreadstat
import pandas as pd

df_path = "anes_timeseries_2020_stata_20210324.dta"

df, meta = pyreadstat.read_dta(df_path)
print(df['V201032'].unique())


[ 1. -1.  2. -9. -8.]


In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal, bernoulli, beta, norm
from scipy.special import expit as logistic_sigmoid
import statsmodels.api as sm
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss


In [53]:
# Keep only observations from respondents who say they indend to vote
print("Shape before filtering:", df.shape)
df = df[df['V201032'].isin([1.])]
print("Shape after filtering:", df.shape)

# Define the outcome of interest as 1 if the voter will vote for the Republicans, 0 otherwise. (Hint: in the codebook look for variable V201033 )
Y = df['V201033'].apply(lambda x: 1 if x == '2. Donald Trump' else 0)

# Drop missing values separately from X and Y
df = df.dropna()
# Y = Y[X.index]
# Drop missing values -- stacking here helps ensure the same misisn values are removed for both X and Y. 
# YX = np.hstack((np.array(Y)[:, np.newaxis], X))
# YX = YX[~np.any(np.isnan(YX), axis=1)]


# Create a design matrix. 
# Turn these features into a numeric matrix so we can use them to fit the model.
# Age: 
age_num = pd.to_numeric(df['V201507x'], errors='coerce')

# College Degree or Higher:
college_edu = df['V201510'].apply(lambda x: 1 if x in ["6. Bachelor's degree (e.g. BA, AB, BS)",
                                                       "7. Master's degree (e.g. MA, MS, MEng, MEd, MSW, MBA)",
                                                       "4. Associate degree in college - occupational/vocational",
                                                       "5. Associate degree in college - academic"] else 0)

# Trust in media
trust_media = df['V201377'].apply(lambda x: 1 if x == 5 else 0)

# Create design matrix 
X = pd.DataFrame({
    'Age': age_num,
    'College_Degree_or_Higher': college_edu,
    'Trust_in_Media': trust_media
})
print(X)
# Check for missing values 
missing_values_age = df['V201507x'].isnull().sum()
missing_values_college_edu = df['V201510'].isnull().sum()
missing_values_trust_media = df['V201377'].isnull().sum()
print("Missing values in 'V201507x' column:", missing_values_age)
print("Missing values in 'V201510' column:", missing_values_college_edu)
print("Missing values in 'V201377' column:", missing_values_trust_media)


# Define relevant design matrix (with associated names) and outcome to model
X_with_intercept = sm.add_constant(X)

Shape before filtering: (8280, 1381)
Shape after filtering: (7272, 1381)
Empty DataFrame
Columns: [Age, College_Degree_or_Higher, Trust_in_Media]
Index: []
Missing values in 'V201507x' column: 0
Missing values in 'V201510' column: 0
Missing values in 'V201377' column: 0


In [47]:
# Check for missing values in X
print("Missing values in X:")
print(X.isnull().sum())

# Check the number of observations remaining after dropping missing values
print("Number of observations remaining after dropping missing values:")
print(X.shape[0])

# Fit the logistic regression model only if there are observations remaining
if X.shape[0] > 0:
    model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()
    print(model.summary())
else:
    print("No observations remaining after dropping missing values.")


Missing values in X:
Age                         0
College_Degree_or_Higher    0
Trust_in_Media              0
dtype: int64
Number of observations remaining after dropping missing values:
0
No observations remaining after dropping missing values.


In [49]:
import statsmodels.api as sm

# Fit the logistic regression model
model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()

# Print the summary of the model
print(model.summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [35]:
# Fit the logistic regression model
model = sm.Logit(Y, X_with_intercept[['const', 'Trust_in_Media']]).fit()

# Print the summary of the model
print(model.summary())

ValueError: zero-size array to reduction operation maximum which has no identity

In [ ]:
# Get summary results
summary = model.summary()
print(summary)